#  importing request

In [108]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyp
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier ,RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime , timedelta
import pytz

# using openweathermap 

In [109]:
API_KEY="2496c8d59c15b86e2865bdfddad9ad67"  # open_wearhermap_api
BASE_URL="http://api.openweathermap.org/data/2.5/weather?q="# base url for making actual request

# 1. Fetch Current Weather data

In [168]:
def get_current_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid=2496c8d59c15b86e2865bdfddad9ad67&units=metric"

    response = requests.get(url)
    data = response.json()

    # Check if the API returned an error
    if response.status_code != 200 or 'name' not in data:
        print(f"API Error: {data.get('message', 'Unknown error')}")
        return None

    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_speed': data['wind']['speed'],
        'WindGustDir': data['wind']['deg'],
        'pressure': data['main']['pressure'],
    }


# 2. Historical Data

In [111]:
historical_data=pd.read_csv("weather.csv")

In [112]:
historical_data

,MinTemp,MaxTemp,WindGustDir,WindGustSpeed,Humidity,Pressure,Temp,RainTomorrow
0,8.0,24.3,NW,30.0,29,1015.0,23.6,Yes
1,14.0,26.9,ENE,39.0,36,1008.4,25.7,Yes
2,13.7,23.4,NW,85.0,69,1007.2,20.2,Yes
3,13.3,15.5,NW,54.0,56,1007.0,14.1,Yes
4,7.6,16.1,SSE,50.0,49,1018.5,15.4,No
...,...,...,...,...,...,...,...,...
361,9.0,30.7,NNW,76.0,15,1010.8,30.0,No
362,7.1,28.4,N,48.0,22,1016.9,28.2,No
363,12.5,19.9,ESE,43.0,47,1022.8,18.3,No
364,12.5,26.9,NW,46.0,39,1016.2,25.9,No


In [113]:
historical_data.head()

,MinTemp,MaxTemp,WindGustDir,WindGustSpeed,Humidity,Pressure,Temp,RainTomorrow
0,8.0,24.3,NW,30.0,29,1015.0,23.6,Yes
1,14.0,26.9,ENE,39.0,36,1008.4,25.7,Yes
2,13.7,23.4,NW,85.0,69,1007.2,20.2,Yes
3,13.3,15.5,NW,54.0,56,1007.0,14.1,Yes
4,7.6,16.1,SSE,50.0,49,1018.5,15.4,No


In [114]:
historical_data.size

2928

In [115]:
historical_data.shape

(366, 8)

In [116]:
historical_data.isnull().sum()

MinTemp          0
MaxTemp          0
WindGustDir      3
WindGustSpeed    2
Humidity         0
Pressure         0
Temp             0
RainTomorrow     0
dtype: int64

In [117]:
historical_data=historical_data.dropna()

In [118]:
historical_data.drop_duplicates(inplace=True)

C:\Users\Strange\AppData\Local\Temp\ipykernel_8028\4097610674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_data.drop_duplicates(inplace=True)


In [119]:
historical_data.corr(numeric_only=True)

,MinTemp,MaxTemp,WindGustSpeed,Humidity,Pressure,Temp
MinTemp,1.000000,0.752368,0.219670,-0.039926,-0.496746,0.722580
MaxTemp,0.752368,1.000000,0.109210,-0.533594,-0.379980,0.989255
WindGustSpeed,0.219670,0.109210,1.000000,-0.069087,-0.526344,0.072418
Humidity,-0.039926,-0.533594,-0.069087,1.000000,-0.010776,-0.582014
Pressure,-0.496746,-0.379980,-0.526344,-0.010776,1.000000,-0.346464
Temp,0.722580,0.989255,0.072418,-0.582014,-0.346464,1.000000


In [120]:
def read_historical_data(filename):
    df=pd.read_csv(filename)
    df=df.dropna()
    df=df.drop_duplicates()
    return df

# 3. Prepare Data for learning

In [121]:
from sklearn.preprocessing import LabelEncoder

def prepare_data(data):
    le = LabelEncoder()
    
    # Encode categorical variables
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

    # Define features and target
    x = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']]
    y = data['RainTomorrow']
    
    return x, y, le


# 4. Train Rain prediction model

In [131]:
def train_rain_model(x,y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(x_train , y_train)

    y_pred=model.predict(x_test)

    print("Min Square error for Rain model")

    print(mean_squared_error(y_test , y_pred))

    return model

# 4. Prepare regrission data

In [141]:
def prepare_regrission_data(data , features):
    x,y =[],[] # initaialize list for features and target values

    for i in range (len(data) -1):
        x.append(data[features].iloc[i])    # present 
        y.append(data[features].iloc[i+1])    # future

    x=np.array(x).reshape(-1,1)
    y=np.array(y)

    return x , y 

# Train regrission Model

In [124]:
def train_regrission_model(x,y):
    model=RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(x,y)
    return model

# Predict feature

In [158]:
def prdict_feature(model , current_value):
    prediction = [current_value]

    for i in range(5):
        next_value = model.predict(np.array([[prediction[-1]]])) 
        prediction.append(next_value[0])
        
    return prediction[1:]   

# Weather Analsis Function

In [169]:
def weather_view():
    city = input('Enter any city name: ')
    current_weather = get_current_weather(city)

    if current_weather is None:
        print("Could not fetch weather data.")
        return

    # Load historical data
    historical_data = read_historical_data("C:\\Users\\Strange\\Documents\\data sc\\projects ml\\waether prediction\\weather prediction ml\\weather.csv")

    # Prepare and train rain prediction model
    x, y, le = prepare_data(historical_data)
    rain_model = train_rain_model(x, y)

    # Map wind direction to compass points
    wind_deg=current_weather['WindGustDir'] % 360
    compass_points = [
        ('N', 348.75, 360.0), ('N', 0.0, 11.25),
        ('NNE', 11.25, 33.75), ('NE', 33.75, 56.25),
        ('ENE', 56.25, 78.75), ('E', 78.75, 101.25),
        ('ESE', 101.25, 123.75), ('SE', 123.75, 146.25),
        ('SSE', 146.25, 168.75), ('S', 168.75, 191.25),
        ('SSW', 191.25, 213.75), ('SW', 213.75, 236.25),
        ('WSW', 236.25, 258.75), ('W', 258.75, 281.25),
        ('WNW', 281.25, 303.75), ('NW', 303.75, 326.25),
        ('NNW', 326.25, 348.75)
    ]

    compass_direction = next(label for label, start, end in compass_points if start <= wind_deg < end)
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir': compass_direction_encoded,
        'WindGustSpeed': current_weather['wind_speed'],
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['pressure'],
        'Temp': current_weather['current_temp'],
    }

    current_df = pd.DataFrame([current_data])

    # Rain prediction
    rain_prediction = rain_model.predict(current_df)[0]

    # Prepare regression models
    x_temp, y_temp = prepare_regrission_data(historical_data, 'Temp')
    x_hum, y_hum = prepare_regrission_data(historical_data, 'Humidity')

    temp_model = train_regrission_model(x_temp, y_temp)
    hum_model = train_regrission_model(x_hum, y_hum)

    # Predict future values
    future_temp=prdict_feature(temp_model , current_weather['temp_min'])
    future_humidity=prdict_feature(temp_model , current_weather['humidity'])

    # Time setup
    timezone = pytz.timezone('Asia/Kolkata')  # More specific
    now = datetime.now(timezone)
    next_hour = now.replace(minute=0, second=0, microsecond=0)
    future_time = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    # Display results
    print(f'\nCity: {city}, {current_weather["country"]}')
    print(f'Current Temperature: {current_weather["current_temp"]}°C')
    print(f'Feels Like: {current_weather["feels_like"]}°C')
    print(f'Maximum Temperature: {current_weather["temp_max"]}°C')
    print(f'Minimum Temperature: {current_weather["temp_min"]}°C')
    print(f'Humidity: {current_weather["humidity"]}%')
    print(f'Weather Description: {current_weather["description"].title()}')
    print(f'Rain Prediction: {"Yes" if rain_prediction else "No"}')

    print("\nFuture Temperature Prediction:")
    for time, temp in zip(future_time, future_temp):
        print(f'{time}: {round(temp, 1)}°C')

    print("\nFuture Humidity Prediction:")
    for time, humidity in zip(future_time, future_humidity):
        print(f'{time}: {round(humidity, 1)}%')

# Call the function
weather_view()


Enter any city name:  Guwahati


Min Square error for Rain model
0.1506849315068493

City: Guwahati, IN
Current Temperature: 25°C
Feels Like: 25°C
Maximum Temperature: 25°C
Minimum Temperature: 25°C
Humidity: 83%
Weather Description: Clear Sky
Rain Prediction: Yes

Future Temperature Prediction:
02:00: 26.7°C
03:00: 24.4°C
04:00: 24.4°C
05:00: 24.4°C
06:00: 24.4°C

Future Humidity Prediction:
02:00: 33.1%
03:00: 25.1%
04:00: 20.1%
05:00: 16.6%
06:00: 14.4%
